# Data Cleaning

## Creating a Full California Dataset by merging Residential and Business Data:

In [1]:
import csv
import pandas as pd
import re
path = r'Data_Folder/Cleaned_Data/'
path1 = r'Data_Folder/Others/'

First, I read in the Residential and Business csv files, drop duplicate columns, and do a one-to-one merge.

In [2]:
"""Reading in csv files and dropping duplicate columns"""
CA_Cn = pd.read_csv(path + 'CA_Bus_Cn.csv')
CA_Cn = CA_Cn.drop(['Unnamed: 0', 'Unnamed: 0.1'],1)

Res_Cn = pd.read_csv(path + 'CA_Res_Cn.csv')
Res_Cn = Res_Cn.drop(['Unnamed: 0'],1)
"""Merging two datasets one-toCA_Cn = CA_Cn.reset_index()
-one"""
merge_list = ['Material Category', 'Material Type', 'Jurisdiction(s)']
CA_Cn = CA_Cn.merge(Res_Cn, left_on=merge_list, right_on = merge_list)
CA_Cn = CA_Cn.sort_values('Jurisdiction(s)')
CA_Cn = CA_Cn.reset_index(drop=True)
CA_Cn.to_csv('CA_Cn.csv')

## Creating a Dataset with county characteristics:
First I clean voting data and income data, and then merge one-to-one the two datasets.

In [3]:
vote = pd.read_csv(path1 + 'CA_VoteReg.csv')
""" Removes the superfluous rows, drop rows
with missing values, and change strings to floats."""
vote.columns = vote.iloc[2]
vote = vote.reindex(vote.index.drop([0,2]))
vote = vote.dropna()
for i in vote.columns:
    if i != 'County':
        vote[i] = vote[i].str.replace(",","").astype(float)
vote = vote.reset_index(drop=True)
""" Removes state total row in order to merge """
state_total_id = vote[vote['County']== 'State Total'].index[0]
vote = vote.drop(vote.index[state_total_id])
vote = vote.sort_values('County')

In [4]:
income = pd.read_csv(path1 + 'CountyIncome.csv', dtype={2:'str'})
""" Organize rows and change strings into floats"""
income = income.sort_values('County')
income = income.reset_index()
income = income.drop('index',1)

income_list = ['Returns', 'AGI', 'Median Income',
               'Taxable Assessed', 'Population']
for i in income_list:
    income[i]= income[i].str.replace(',', '')
    income[i]= income[i].str.replace('$', '')
    income[i] = income[i].astype(float)


In [5]:
""" Merge 1-1 voter data set with income data set"""
info = vote.merge(income, left_on=['County'], right_on = ['County'])
info.to_csv('Info.csv')


Now I clean the educational attainment data and merge education to the characteristic data set.

In [6]:
edu = pd.read_csv('Data_Folder/Others/CA_edu.csv', low_memory = False)
""" Only keep columns about counties for educational attainment"""
edu = edu.filter(['ind_definition', 'strata_one_name','geotype',
                  'race_eth_name','reportyear','county_name',
                  'numerator', 'denominator', 'estimate'])
edu = edu.dropna(subset=['county_name'])
edu = edu.sort_values('county_name')
edu=edu[(edu['race_eth_name']== 'Total') & (edu['reportyear']=='2011-2015')]
edu = edu[edu['geotype']=='CO']
edu['strata_one_name'] = edu['strata_one_name'].fillna(0)
edu = edu[edu['strata_one_name']== 0]
edu = edu.reset_index()
edu = edu.drop(['ind_definition', 'strata_one_name', 
                'geotype', 'race_eth_name'],1)
edu = edu.rename(columns={'numerator': 'edu_num', 'denominator': 'edu_den',
                   'estimate': 'edu_perc'})

In [7]:
"""merge education data to the characteristic data set"""
Total_Info = info.merge(edu, left_on=['County'], right_on = ['county_name'])

In [8]:
Total_Info.to_csv(path + 'Total_Info.csv')

## Merge the california business and residential data set with the characteristic data set:


In [9]:
""" Clean to have the same merging variable """
county_clean = re.compile(r'\s\(Countywide\)')
CA_Cn['County'] = CA_Cn['Jurisdiction(s)'].str.replace(county_clean, '')
CA_Cn = CA_Cn.drop('Jurisdiction(s)', 1)

In [10]:
total = pd.merge(CA_Cn, Total_Info, left_on="County", right_on="County", 
                 how= "left", validate = "m:1")

In [11]:
total = total.reindex(columns=(['County']+
                               list([a for a in total.columns if
                                     a != 'County'])))

## Create percentage columns for analysis: 

In [48]:
""" Create percentage columns """
total['Total Waste'] = total['Total Disposed Tons'] + total['Total Curbside Recycle Tons']\
                    + total['Total Curbside Organics Tons'] + \
                    total['Total Other Diversion Tons']
total['Disp_perc'] = (total['Total Disposed Tons']/ 
                   total['Total Waste']) * 100
total['Rec_perc'] = (total['Total Curbside Recycle Tons']/ 
                  total['Total Waste']) * 100
total['Recycling Totals'] = total.groupby('Material Category') \
                        ['Material Tons in Curbside Recycle'].transform('sum')
total['Disposal Totals'] = total.groupby('Material Category') \
                        ['Material Tons Disposed'].transform('sum')
total['Compost Totals'] = total.groupby('Material Category') \
                        ['Material Tons in Curbside Organics'].\
                        transform('sum')
total['Other Totals'] = total.groupby('Material Category') \
                        ['Material Tons in Other Diversion'].transform('sum')
total.to_csv('Complete_CA.csv')

## Create California Data Set with no Organic Materials:
<br>
Organic materials are not recyclable and skew how well counties recycle their waste. For example, some counties produce many organic waste from being agricultural and so they would have low recycling percentage.

In [45]:
no_org = total[total['Material Category']!= 'Other Organic']
no_org = no_org[no_org.columns.drop(list(no_org.filter(regex='^Total')))]
no_org['material_generated'] = no_org['Material Tons Generated (Sum of all Streams)']
no_org['Total Waste'] = no_org.groupby('County')['material_generated'].\
                        transform('sum')
no_org['Total_rec'] = no_org.groupby('County')\
                    ['Material Tons in Curbside Recycle'].transform('sum')
no_org['Total_disp'] = no_org.groupby('County')\
                    ['Material Tons Disposed'].transform('sum')

## Create percentage columns for analysis:

In [ ]:
no_org['Rec_perc'] = (no_org['Total_rec'] / no_org['Total Waste']) * 100
no_org['Disp_perc'] = (no_org['Total_disp'] / no_org['Total Waste']) * 100
no_org['DV_perc'] = (no_org['Democratic']/ no_org['Registered']) * 100
no_org['Rep_perc'] =(no_org['Republican']/ no_org['Registered']) * 100
no_org['Green_perc'] = (no_org['Green']/ no_org['Registered']) * 100
no_org.reset_index(drop=True)
no_org.to_csv('CA_No_Organic.csv')